In [1]:
from cryosparc.tools import CryoSPARC
import json
import numpy as np

In [2]:
with open('/u/rposert/instance-info.json', 'r') as f:
    instance_info = json.load(f)

cs = CryoSPARC(**instance_info)
assert cs.test_connection()

Connection succeeded to CryoSPARC command_core at http://cryoem0.sbi:40002
Connection succeeded to CryoSPARC command_vis at http://cryoem0.sbi:40003
Connection succeeded to CryoSPARC command_rtp at http://cryoem0.sbi:40005


In [3]:
project_number = 'P312'
job_number = 'J135'
project = cs.find_project(project_number)
refinement = project.find_job(job_number)
particles = refinement.load_output('particles')
hist, bins = np.histogram(particles['alignments3D/alpha'])
print(*hist, sep = '\n')

172
6345
45697
119715
133897
70323
18647
2645
219
10


In [4]:
scale_threshold = 0.9
filtered_particles = particles.query(
    lambda row: row['alignments3D/alpha'] >= scale_threshold
)
print(*np.histogram(filtered_particles['alignments3D/alpha'], bins = bins)[0], sep = '\n')

0
0
0
85802
133897
70323
18647
2645
219
10


In [5]:
workspace_number = 'W1'
cs.save_external_result(
    project_number,
    workspace_number,
    filtered_particles,
    type = 'particle',
    passthrough = (job_number, 'particles'),
    title = f"Filtered per-particle scale >= {scale_threshold}"
)

'J335'